1. 給予文章內容
2. 對於每篇文章給予分類: A,B,C
3. 建立＆測試模型
4. 預測分類
5. 預測率太低的處理

In [17]:
import cPickle as pickle
import numpy as np
from sklearn.datasets.base import Bunch
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB #導入多項式貝葉斯算法
from sklearn import metrics

# Bunch類提供一種key,value的對象形式
# target_name:所有分類集名稱列表
# label:每個文件的分類標籤列表
# filenames:文件路徑  ==> 這是使用檔案形式才需要的
# contents:分詞後文件詞向量形式
bunch = Bunch(target_name=[],label=[],contents=[])


In [18]:
# create training data - news data
news1 = "藝人, 八卦，音樂,歌劇,愛不愛"
news2 = "當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡，不只周亭羽也愛小內，但仍把希望放在正宮小內身上"
news3 = "台灣的政治，國台辦不負責，4名副主任排名依序為"
news4 = "於1981年起擔任中國駐聯合國日內瓦辦事處譯員，1987年進入中國外交部國際司"
news5 = "NBA, 三巨頭55分 雷霆拒絕被逆轉,三分火炮穩住軍心"
news6 = "NBA,詹姆斯歸隊 騎士仍不敵公牛吞4連敗,每場輸對手10.5分，可見得整個球隊還處在磨合期"

In [19]:
labelList = np.array(['a','b','c'])
bunch.target_name.extend(labelList)
# for mylabel in labelList:
bunch.label.append(labelList[0])
bunch.contents.append(news1)
bunch.label.append(labelList[0])
bunch.contents.append(news2)
bunch.label.append(labelList[1])
bunch.contents.append(news4)
bunch.label.append(labelList[1])
bunch.contents.append(news3)
bunch.label.append(labelList[2])
bunch.contents.append(news5)
bunch.label.append(labelList[2])
bunch.contents.append(news6)

#對象持久化                                                                                              
file_obj = open("train_set.dat", "wb")
pickle.dump(bunch,file_obj)                      
file_obj.close()

print "構建文本對象結束！！！"

構建文本對象結束！！！


In [60]:
#引入持久化類
import cPickle as pickle
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer  

# 讀取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

#計算訓練語料的tfidf權值並持久化為詞袋

#讀取bunch對象
def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj) 
    file_obj.close()
    return bunch

#寫入bunch對象
def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj) 
    file_obj.close()

# 構建tf-idf詞向量空間對象
tfidfspace = Bunch(target_name=bunch.target_name,label=bunch.label,tdm=[],vocabulary={})
# 使用TfidfVectorizer初始化向量空間模型 
stopword_path = "hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df =1, max_features=18)
# vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df =1,ngram_range=(1,2))
transformer=TfidfTransformer() # 該類會統計每個詞語的tf-idf權值
# 文本轉為詞頻矩陣,單獨保存字典文件 
tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_

# 創建詞袋的持久化
space_path = "tfdifspace.dat"        # 詞向量空間保存路徑
writebunchobj(space_path,tfidfspace)

print "if-idf詞向量空間創建成功！！！"

if-idf詞向量空間創建成功！！！


In [66]:
from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(stop_words=stpwrdlst ,decode_error='ignore')
# X_train_counts = count_vect.fit_transform(tfidfspace.vocabulary)
# X_train_counts.shape

tfidfspace.vocabulary
# 三巨頭55分 : \u4e09\u5de8\u982d55\u5206
# 表示斷詞不夠好. 需要建立 vocabulary 以及 token 

{u'1987\u5e74\u9032\u5165\u4e2d\u570b\u5916\u4ea4\u90e8\u570b\u969b\u53f8': 0,
 u'nba': 1,
 u'\u4e09\u5de8\u982d55\u5206': 2,
 u'\u4e0d\u53ea\u5468\u4ead\u7fbd\u4e5f\u611b\u5c0f\u5167': 3,
 u'\u4f46\u4ecd\u628a\u5e0c\u671b\u653e\u5728\u6b63\u5bae\u5c0f\u5167\u8eab\u4e0a': 4,
 u'\u516b\u5366': 5,
 u'\u53ef\u898b\u5f97\u6574\u500b\u7403\u968a\u9084\u8655\u5728\u78e8\u5408\u671f': 6,
 u'\u53f0\u7063\u7684\u653f\u6cbb': 7,
 u'\u570b\u53f0\u8fa6\u4e0d\u8ca0\u8cac': 8,
 u'\u611b\u4e0d\u611b': 9,
 u'\u65bc1981\u5e74\u8d77\u64d4\u4efb\u4e2d\u570b\u99d0\u806f\u5408\u570b\u65e5\u5167\u74e6\u8fa6\u4e8b\u8655\u8b6f\u54e1': 10,
 u'\u66fe\u5728\u8a18\u8005\u6703\u4e0a\u8868\u660e\u96d6\u7136\u81ea\u5df12\u500b\u90fd\u5f88\u559c\u6b61': 11,
 u'\u6b4c\u5287': 12,
 u'\u6bcf\u5834\u8f38\u5c0d\u624b10': 13,
 u'\u7576\u5e74\u4e5d\u628a\u5200\u5288\u817f\u66dd\u5149\u5f8c': 14,
 u'\u85dd\u4eba': 15,
 u'\u8a79\u59c6\u65af\u6b78\u968a': 16,
 u'\u96f7\u9706\u62d2\u7d55\u88ab\u9006\u8f49': 17}

In [50]:
tfidfspace.tdm.shape
# tfidfspace.label

(6, 12)

In [23]:
tfidfspace.tdm.data

array([ 0.4472136 ,  0.4472136 ,  0.4472136 ,  0.4472136 ,  0.4472136 ,
        0.5       ,  0.5       ,  0.5       ,  0.5       ,  0.70710678,
        0.70710678,  0.57735027,  0.57735027,  0.57735027,  0.42790272,
        0.52182349,  0.52182349,  0.52182349,  0.34430007,  0.4198708 ,
        0.4198708 ,  0.4198708 ,  0.4198708 ,  0.4198708 ])

In [61]:
# create test data - news data
tnews1 = "藝人, 八卦，音樂,歌劇,愛不愛"
tnews2 = "劈腿曝光,也愛小內，但仍希望, 當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡"
tnews3 = "台灣的政治，國台辦不負責"
tnews4 = "中國外交部國際司"
tnews5 = "NBA, 三巨頭55分 雷霆三分"
tnews6 = "NBA,詹姆斯不敵,磨合期"       
# 上面的測試文字，太短, 會有維度的問題
# tnews1 = "藝人, 八卦，音樂,歌劇,愛不愛"
# tnews2 = "當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡，不只周亭羽也愛小內，但仍把希望放在正宮小內身上"
# tnews3 = "台灣的政治，國台辦不負責，4名副主任排名依序為"
# tnews4 = "於1981年起擔任中國駐聯合國日內瓦辦事處譯員，1987年進入中國外交部國際司"
# tnews5 = "NBA, 三巨頭55分 雷霆拒絕被逆轉,三分火炮穩住軍心"
# tnews6 = "NBA,詹姆斯歸隊 騎士仍不敵公牛吞4連敗,每場輸對手10.5分，可見得整個球隊還處在磨合期"

In [46]:
testbunch = Bunch(target_name=[],label=[],contents=[])
testbunch.target_name.extend(labelList)
# for mylabel in labelList:
testbunch.label.append(labelList[0])
testbunch.contents.append(tnews1)
testbunch.label.append(labelList[0])
testbunch.contents.append(tnews2)
testbunch.label.append(labelList[1])
testbunch.contents.append(tnews4)
testbunch.label.append(labelList[1])
testbunch.contents.append(tnews3)
testbunch.label.append(labelList[2])
testbunch.contents.append(tnews5)
testbunch.label.append(labelList[2])
testbunch.contents.append(tnews6)

#對象持久化                                                                                              
file_obj = open("test_set.dat", "wb")
pickle.dump(testbunch,file_obj)                      
file_obj.close()

print "構建文本測試set結束！！！"

構建文本測試set結束！！！


In [62]:
#test詞向量空間創建
# 構建tf-idf詞向量空間對象
testspace = Bunch(target_name=testbunch.target_name,label=testbunch.label,tdm=[],vocabulary={})
# 使用TfidfVectorizer初始化向量空間模型 
stopword_path = "hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,max_df = 0.5, max_features=18)
# vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df = 1, ngram_range=(1,2))

# 補足不足處
# from sklearn.feature_extraction.text import HashingVectorizer
# hv = HashingVectorizer(n_features=23)


transformer=TfidfTransformer() # 該類會統計每個詞語的tf-idf權值
# 文本轉為詞頻矩陣,單獨保存字典文件 
testspace.tdm = vectorizer.fit_transform(testbunch.contents)
testspace.vocabulary = vectorizer.vocabulary_
# 新增
# testspace.tdm = hv.transform(testbunch.contents)

# 創建詞袋的持久化
space_path = "testspace.dat"        # 詞向量空間保存路徑
writebunchobj(space_path,testspace)

print "test詞向量空間創建成功！！！"

test詞向量空間創建成功！！！


In [63]:
print testspace.label
print testspace.tdm.shape
print testspace.tdm.todense()
print type(testspace.tdm.data)
# testspace.tdm.data = (testspace.tdm.data.flatten())
# testspace.tdm.data.reshape(6,23)
print testspace.tdm.shape

['a', 'a', 'b', 'b', 'c', 'c']
(6, 18)
[[ 0.          0.          0.          0.          0.          0.4472136
   0.          0.          0.          0.4472136   0.          0.4472136
   0.          0.          0.4472136   0.          0.          0.4472136 ]
 [ 0.          0.          0.          0.4472136   0.4472136   0.
   0.4472136   0.          0.          0.          0.4472136   0.
   0.4472136   0.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.70710678  0.70710678  0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.50161301  0.61171251  0.          0.          0.          0.          0.
   0.          0.          0.          0.

In [28]:
#預測
#計算分類精度：
def metrics_result(actual,predict):
    print '精度:{0:.3f}'.format(metrics.precision_score(actual,predict,average='macro'))  
    print '召回:{0:0.3f}'.format(metrics.recall_score(actual,predict,average='macro'))  
    print 'f1-score:{0:.3f}'.format(metrics.f1_score(actual,predict,average='macro'))  

In [64]:
# 導入訓練集
trainpath = "tfdifspace.dat"
train_set = readbunchobj(trainpath)

# 導入測試集
testpath = "testspace.dat"
test_set = readbunchobj(testpath)

xlabel = np.array(train_set.label)

print train_set.tdm.shape
print test_set.tdm.shape
print type(train_set.label), train_set.label
print type(xlabel), xlabel.shape

(6, 18)
(6, 18)
<type 'list'> ['a', 'a', 'b', 'b', 'c', 'c']
<type 'numpy.ndarray'> (6,)


In [65]:


# 應用樸素貝葉斯算法 
# 1. 輸入詞袋向量和分類標籤
#alpha 平滑參數:0.001 alpha越小，迭代次數越多，精度越高

clf = MultinomialNB(alpha = 0.001).fit(train_set.tdm, train_set.label)

predicted = clf.predict(test_set.tdm)
total = len(predicted);rate = 0
for flabel,expct_cate in zip(test_set.label,predicted):
    if flabel != expct_cate:
        rate += 1
        print "實際類別:",flabel," -->預測類別:",expct_cate
# 精度
print "error rate:",float(rate)*100/float(total),"%"
print "預測完畢!!!"

metrics_result(test_set.label,predicted)

實際類別: b  -->預測類別: c
error rate: 16.6666666667 %
預測完畢!!!
精度:0.889
召回:0.833
f1-score:0.822
